In [2]:
import os
import matplotlib.pyplot as plt
import copy
import time

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms, utils
from torchvision import datasets, models, transforms
torch.__version__

import pandas as pd
import numpy as np
from PIL import Image
from skimage import io, transform

In [3]:
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"
data = pd.read_csv('train/category_train.csv')

In [47]:
Category = np.array(data.Category.unique())

class Dataset_Category(Dataset):
    def __init__(self, csv_file, transform=None):
        self.df = pd.read_csv(csv_file)
        self.transform = transform
        self.category = Category

    def __len__(self):
        return 20

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        path = "Myauto_data/Car_Images/Car_Images{}".format(self.df.iloc[idx].img_path)
        image = transform.resize(io.imread(path), (224, 224))/255
        image = image[:, :, :3]
        
        y = np.where(self.category == self.df.iloc[idx].Category)[0]
        sample = (torch.Tensor(np.einsum('ijk->kij',image)), torch.Tensor(y).long())

        return sample

In [48]:
train = Dataset_Category('./train/category_train.csv', 'Myauto_data/Car_Images/Car_Images/')
val = Dataset_Category('./test/category_test.csv', 'Myauto_data/Car_Images/Car_Images/')

train_loader = DataLoader(train, batch_size=4,shuffle=True, num_workers=4)
val_loader = DataLoader(val, batch_size=4,shuffle=True, num_workers=4)

dataloaders = {'train': train_loader, 'val': val_loader}
dataset_sizes = {'train': len(train_loader), 'val': len(val_loader)}

In [49]:
class model_init(nn.Module):
    def __init__(self):
        super(model_init, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(models.resnet18(pretrained=True))
        self.layers.append(nn.Linear(1000, 512)) 
        self.layers.append(nn.Dropout(0.1))
        self.layers.append(nn.Linear(512, 64))
        self.layers.append(nn.Dropout(0.1))
        self.layers.append(nn.Linear(64, 4))
        self.layers.append(nn.Softmax())
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [77]:
def train_helper(category, model, optimization, judge, timeout):
    if category == 'train':
        model.train()
    else:
        model.eval()

    loss_tmp = 0.0
    correct_tmp = 0

    for inputs, labels in dataloaders[category]:
        inputs = inputs.to(device)
        labels = labels.to(device)
        labels = labels.reshape((labels.shape[0]))
        optimization.zero_grad()

        with torch.set_grad_enabled(category == 'train'):
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            loss = judge(outputs, labels.reshape((-1,)))

            if category == 'train':
                loss.backward()
                optimization.step()

        # statistics
        loss_tmp += loss.item() * inputs.size(0)
        correct_tmp += torch.sum(preds == labels.data)
    if category == 'train':
        timeout.step()

    epoch_loss = loss_tmp / dataset_sizes[category]
    epoch_acc = correct_tmp.double() / dataset_sizes[category]

    print('{} Loss: {:.4f} Accuracy: {:.4f}'.format(category, epoch_loss, epoch_acc/4))
    
    return epoch_acc, copy.deepcopy(model.state_dict())

In [78]:
def train_model(model, judge, optimization, timeout, epochs=20):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_accuracy = 0.0

    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch + 1, epochs))
        print('-----------------------------------------')
        train_helper('train', model, optimization, judge, timeout)
        accuracy_tmp, weight_tmp = train_helper('val', model, optimization, judge, timeout)
        if accuracy_tmp > best_accuracy:
            best_accuracy = accuracy_tmp
            best_model_weights = weight_tmp

        print()
        
    print('Best val Acc: {:4f}'.format(best_accuracy))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [79]:
criterion = nn.CrossEntropyLoss()
model = model_init()
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [80]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler, epochs=4)

Epoch 1/4
-----------------------------------------


/Users/andghuladze/Python/venv_3.7/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


train Loss: 5.6502 Accuracy: 0.1500
val Loss: 5.3888 Accuracy: 0.4500

Epoch 2/4
-----------------------------------------
train Loss: 4.7934 Accuracy: 0.6000
val Loss: 5.2217 Accuracy: 0.4500

Epoch 3/4
-----------------------------------------
train Loss: 4.0186 Accuracy: 0.8000
val Loss: 5.1702 Accuracy: 0.4500

Epoch 4/4
-----------------------------------------
train Loss: 3.7554 Accuracy: 0.8000
val Loss: 5.5611 Accuracy: 0.2500

Best val Acc: 1.800000


In [ ]:
torch.save(model.state_dict(), 'models/category_model.pt')